In [54]:
from google.colab import drive
import json

# Monta Google Drive en el entorno de Colab
drive.mount('/content/drive')

# Ruta al archivo de contraseñas JSON en drive
file_path = '/content/drive/MyDrive/pass.json'

# Lee el archivo JSON
with open(file_path, 'r') as file:
    config_data = json.load(file)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
# Leer los datos de la API
import requests
import pandas as pd
from datetime import datetime, timedelta

# Acceder a las contraseñas de la api
api_key = config_data['api_key']

# Lista de nombres de países
paises = [
    'Afghanistan', 'Albania', 'Algeria', 'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Aruba', 'Austria',
    'Barbados', 'Belgium', 'Botswana', 'Brazil', 'Bulgaria', 'Canada', 'Colombia',
    'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Federated States of Micronesia', 'Finland',
    'France', 'Germany', 'Greece', 'Guyana', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Ireland', 'Israel',
    'Italy', 'Jamaica', 'Kenya', 'Kuwait', 'Latvia', 'Lithuania', 'Luxembourg', 'Madagascar', 'Malawi', 'Malta',
    'Marshall Islands', 'Mexico', 'Moldova', 'Mongolia', 'Montenegro', 'Myanmar', 'Netherlands', 'New Zealand',
    'North Macedonia', 'Norway', 'Oman', 'Palau', 'Papua New Guinea', 'Paraguay', 'Philippines', 'Poland',
    'Portugal', 'Romania', 'Russia', 'Saint Lucia', 'Saint Vincent and the Grenadines', 'Samoa', 'Senegal', 'Serbia',
    'Slovakia', 'Slovenia', 'Solomon Islands', 'Spain', 'Suriname', 'Sweden', 'Switzerland',
    'Tanzania', 'Thailand', 'Tonga', 'Trinidad and Tobago', 'United Kingdom of Great Britain and Northern Ireland',
    'United States', 'Vanuatu', 'Zimbabwe'
]

# Lista para almacenar los datos de cada país
all_data = []
# Lista para almacenar los nombres de los países
countries = []
id = []

# Iterar sobre la lista de países
for i,pais in enumerate(paises):
    # Hacer la solicitud a la API para cada país
    url = f'https://api.openweathermap.org/data/2.5/weather?q={pais}&units=metric&lang=es&appid={api_key}'
    response = requests.get(url)

    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        # Convertir la respuesta a JSON y agregar los datos al listado
        data_json = json.loads(response.text)
        all_data.append({
            'main': data_json['main'],
            'weather': data_json['weather'][0]['description'],
            'clouds': data_json['clouds'],
            'wind': data_json['wind'],
            'visibility' : data_json['visibility'],
            'coord': data_json['coord'],
            'timezone': data_json['timezone'],
            'name': data_json['name'],
            'id': i + 1,
        })
    else:
        print(f"No se pudo obtener datos para {pais}. Código de estado: {response.status_code}")

In [56]:
# Convertir la lista de datos a un DataFrame
df = pd.json_normalize(all_data, sep='_')

# Mostrar el DataFrame
df

,weather,visibility,timezone,name,id,main_temp,main_feels_like,main_temp_min,main_temp_max,main_pressure,main_humidity,main_sea_level,main_grnd_level,clouds_all,wind_speed,wind_deg,wind_gust,coord_lon,coord_lat
0,cielo claro,10000,16200,Afghanistan,1,-0.87,-4.71,-0.87,-0.87,1025,42,1025.0,779.0,0,3.19,40,3.10,66.0000,33.0000
1,algo de nubes,10000,3600,Albania,2,12.69,12.03,12.69,12.69,1013,77,1013.0,1002.0,21,2.24,297,2.40,20.0000,41.0000
2,algo de nubes,10000,3600,Algeria,3,22.63,21.24,22.63,22.63,1019,11,1019.0,961.0,18,2.93,36,3.00,3.0000,28.0000
3,lluvia moderada,10000,-14400,Anguilla,4,21.19,21.98,20.95,21.86,1015,100,NaN,NaN,75,2.06,210,NaN,-63.0500,18.2167
4,lluvia ligera,10000,-14400,Antigua and Barbuda,5,21.55,21.33,20.69,21.55,1015,60,NaN,NaN,75,3.09,30,NaN,-61.8000,17.0500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,muy nuboso,10000,-14400,Trinidad and Tobago,77,31.08,38.08,31.08,31.08,1014,79,1014.0,1014.0,67,9.19,127,10.80,-61.0000,11.0000
77,nieve,7868,0,United Kingdom,78,0.84,-5.75,0.75,3.80,989,88,989.0,980.0,100,9.48,94,17.46,-2.6953,54.7584
78,algo de nubes,10000,-21600,United States of America,79,8.00,4.97,8.00,8.00,1001,42,1001.0,937.0,21,5.29,247,9.30,-98.5000,39.7600
79,lluvia de gran intensidad,10000,39600,Vanuatu,80,27.09,30.67,27.09,27.09,1001,87,1001.0,1001.0,100,4.93,33,5.00,167.0000,-16.0000


In [57]:
# Convertir el cambio de horario de segundos a timestamp UTC
for idx, i in enumerate(df['timezone']):
    fecha_hora = datetime.utcnow() + timedelta(seconds=i)
    fecha_hora_formateada = fecha_hora.strftime("%Y/%m/%d %H:%M")
    fecha_hora_formateada = pd.to_datetime(fecha_hora_formateada)
    df.at[idx, 'timezone'] = fecha_hora_formateada

# Mostrar el DataFrame corregido
df

,weather,visibility,timezone,name,id,main_temp,main_feels_like,main_temp_min,main_temp_max,main_pressure,main_humidity,main_sea_level,main_grnd_level,clouds_all,wind_speed,wind_deg,wind_gust,coord_lon,coord_lat
0,cielo claro,10000,2024-02-08 20:55:00,Afghanistan,1,-0.87,-4.71,-0.87,-0.87,1025,42,1025.0,779.0,0,3.19,40,3.10,66.0000,33.0000
1,algo de nubes,10000,2024-02-08 17:25:00,Albania,2,12.69,12.03,12.69,12.69,1013,77,1013.0,1002.0,21,2.24,297,2.40,20.0000,41.0000
2,algo de nubes,10000,2024-02-08 17:25:00,Algeria,3,22.63,21.24,22.63,22.63,1019,11,1019.0,961.0,18,2.93,36,3.00,3.0000,28.0000
3,lluvia moderada,10000,2024-02-08 12:25:00,Anguilla,4,21.19,21.98,20.95,21.86,1015,100,NaN,NaN,75,2.06,210,NaN,-63.0500,18.2167
4,lluvia ligera,10000,2024-02-08 12:25:00,Antigua and Barbuda,5,21.55,21.33,20.69,21.55,1015,60,NaN,NaN,75,3.09,30,NaN,-61.8000,17.0500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,muy nuboso,10000,2024-02-08 12:25:00,Trinidad and Tobago,77,31.08,38.08,31.08,31.08,1014,79,1014.0,1014.0,67,9.19,127,10.80,-61.0000,11.0000
77,nieve,7868,2024-02-08 16:25:00,United Kingdom,78,0.84,-5.75,0.75,3.80,989,88,989.0,980.0,100,9.48,94,17.46,-2.6953,54.7584
78,algo de nubes,10000,2024-02-08 10:25:00,United States of America,79,8.00,4.97,8.00,8.00,1001,42,1001.0,937.0,21,5.29,247,9.30,-98.5000,39.7600
79,lluvia de gran intensidad,10000,2024-02-09 03:25:00,Vanuatu,80,27.09,30.67,27.09,27.09,1001,87,1001.0,1001.0,100,4.93,33,5.00,167.0000,-16.0000


In [58]:
# Ordenar la lista de datos
column_order = ['id','name', 'weather', 'main_temp', 'main_feels_like', 'main_temp_min', 'main_temp_max',
                'main_pressure', 'main_humidity', 'main_sea_level', 'main_grnd_level', 'clouds_all',
                'wind_speed', 'wind_deg', 'wind_gust','visibility','coord_lon',	'coord_lat','timezone']
df = df[column_order]

# Mostrar el DataFrame
df

,id,name,weather,main_temp,main_feels_like,main_temp_min,main_temp_max,main_pressure,main_humidity,main_sea_level,main_grnd_level,clouds_all,wind_speed,wind_deg,wind_gust,visibility,coord_lon,coord_lat,timezone
0,1,Afghanistan,cielo claro,-0.87,-4.71,-0.87,-0.87,1025,42,1025.0,779.0,0,3.19,40,3.10,10000,66.0000,33.0000,2024-02-08 20:55:00
1,2,Albania,algo de nubes,12.69,12.03,12.69,12.69,1013,77,1013.0,1002.0,21,2.24,297,2.40,10000,20.0000,41.0000,2024-02-08 17:25:00
2,3,Algeria,algo de nubes,22.63,21.24,22.63,22.63,1019,11,1019.0,961.0,18,2.93,36,3.00,10000,3.0000,28.0000,2024-02-08 17:25:00
3,4,Anguilla,lluvia moderada,21.19,21.98,20.95,21.86,1015,100,NaN,NaN,75,2.06,210,NaN,10000,-63.0500,18.2167,2024-02-08 12:25:00
4,5,Antigua and Barbuda,lluvia ligera,21.55,21.33,20.69,21.55,1015,60,NaN,NaN,75,3.09,30,NaN,10000,-61.8000,17.0500,2024-02-08 12:25:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,77,Trinidad and Tobago,muy nuboso,31.08,38.08,31.08,31.08,1014,79,1014.0,1014.0,67,9.19,127,10.80,10000,-61.0000,11.0000,2024-02-08 12:25:00
77,78,United Kingdom,nieve,0.84,-5.75,0.75,3.80,989,88,989.0,980.0,100,9.48,94,17.46,7868,-2.6953,54.7584,2024-02-08 16:25:00
78,79,United States of America,algo de nubes,8.00,4.97,8.00,8.00,1001,42,1001.0,937.0,21,5.29,247,9.30,10000,-98.5000,39.7600,2024-02-08 10:25:00
79,80,Vanuatu,lluvia de gran intensidad,27.09,30.67,27.09,27.09,1001,87,1001.0,1001.0,100,4.93,33,5.00,10000,167.0000,-16.0000,2024-02-09 03:25:00


In [59]:
# Ver la informacion del dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               81 non-null     int64  
 1   name             81 non-null     object 
 2   weather          81 non-null     object 
 3   main_temp        81 non-null     float64
 4   main_feels_like  81 non-null     float64
 5   main_temp_min    81 non-null     float64
 6   main_temp_max    81 non-null     float64
 7   main_pressure    81 non-null     int64  
 8   main_humidity    81 non-null     int64  
 9   main_sea_level   61 non-null     float64
 10  main_grnd_level  61 non-null     float64
 11  clouds_all       81 non-null     int64  
 12  wind_speed       81 non-null     float64
 13  wind_deg         81 non-null     int64  
 14  wind_gust        66 non-null     float64
 15  visibility       81 non-null     int64  
 16  coord_lon        81 non-null     float64
 17  coord_lat        8

In [60]:
# Conectarse a redshift
import psycopg2
from sqlalchemy import create_engine

# Conecta con Redshift
redshift_credentials = config_data['redshift_credentials'] # busca del archivo pass.json montado
conn = psycopg2.connect(
    dbname=redshift_credentials['dbname'],
    user=redshift_credentials['user'],
    password=redshift_credentials['password'],
    host=redshift_credentials['host'],
    port=redshift_credentials['port']
)

# Crea un motor para SQLAlchemy
engine = create_engine(f"postgresql://{redshift_credentials['user']}:{redshift_credentials['password']}@{redshift_credentials['host']}:{redshift_credentials['port']}/{redshift_credentials['dbname']}")

In [61]:
# Define la estructura de la tabla en redshift, dropear si existe y luego crearla
tabla_sql = '''
DROP TABLE IF EXISTS weather_data;
CREATE TABLE weather_data (
    id INT PRIMARY KEY,
    name VARCHAR(100),
    weather VARCHAR(100),
    main_temp DOUBLE PRECISION,
    main_feels_like DOUBLE PRECISION,
    main_temp_min DOUBLE PRECISION,
    main_temp_max DOUBLE PRECISION,
    main_pressure INT,
    main_humidity INT,
    main_sea_level DOUBLE PRECISION,
    main_grnd_level DOUBLE PRECISION,
    clouds_all INT,
    wind_speed DOUBLE PRECISION,
    wind_deg INT,
    wind_gust DOUBLE PRECISION,
    visibility INT,
    coord_lon DOUBLE PRECISION,
    coord_lat DOUBLE PRECISION,
    timezone TIMESTAMP
) DISTSTYLE KEY DISTKEY(id) SORTKEY(id);
'''
try:
    # Ejecuta la consulta SQL para crear la tabla
    cursor = conn.cursor()
    cursor.execute("ROLLBACK")  # Realiza un rollback explícito para limpiar el estado de la transacción abortada
    conn.commit()  # Confirma el rollback
    cursor.execute(tabla_sql)  # Intenta ejecutar la consulta nuevamente
    conn.commit()  # Confirma la creación de la tabla
    print("Tabla creada exitosamente.")
except Exception as e:
    print("Error al crear la tabla:", e)
    conn.rollback()  # Realiza un rollback en caso de error

Tabla creada exitosamente.


In [62]:
# Insertar los datos en las tablas ya creadas
# Crea un cursor
cursor = conn.cursor()

for index, row in df.iterrows():
    # Consulta SQL para la inserción con todas las columnas
    sql_insert = '''
    INSERT INTO weather_data (
        id, name, weather, main_temp, main_feels_like, main_temp_min,
        main_temp_max, main_pressure, main_humidity, main_sea_level,
        main_grnd_level, clouds_all, wind_speed, wind_deg, wind_gust,
        visibility, coord_lon, coord_lat, timezone
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    '''

    # Extrae los valores de cada fila
    values = (
        row['id'],row['name'], row['weather'], row['main_temp'], row['main_feels_like'],
        row['main_temp_min'], row['main_temp_max'], row['main_pressure'],
        row['main_humidity'], row['main_sea_level'], row['main_grnd_level'],
        row['clouds_all'], row['wind_speed'], row['wind_deg'], row['wind_gust'],
        row['visibility'], row['coord_lon'], row['coord_lat'], row['timezone']
    )

    # Ejecuta la consulta SQL con los valores de la fila actual
    cursor.execute(sql_insert, values)

# Confirma la transacción
conn.commit()

# Cierra el cursor y la conexión
cursor.close()
conn.close()